# Zarr 101

In [2]:
from pystac.extensions.storage import StorageExtension
from datetime import datetime
from pystac_client import Client as pystac_client
from odc.stac import configure_rio, stac_load

import xarray

## Context

When dealing with large data files or collections, it's often impossible to load all the data you want to analyze into a single computer's RAM at once. This is a situation where the Pangeo ecosystem can help you a lot. Xarray offers the possibility to work lazily on data __chunks__, which means pieces of an entire dataset. By reading a dataset in __chunks__ we can process our data piece by piece on a single computer and even on a distributed computing cluster using Dask (Cloud or HPC for instance).

How we will process these 'chunks' in a parallel environment will be discussed in [dask_introduction](./dask_introduction.ipynb). The concept of __chunk__ will be explained here.

When we process our data piece by piece, it's easier to have our input or ouput data also saved in __chunks__. [Zarr](https://zarr.readthedocs.io/en/stable/) is the reference library in the Pangeo ecosystem to save our Xarray multidimentional datasets in __chunks__.

[Zarr](https://zarr.readthedocs.io/en/stable/) is not the only file format which uses __chunk__. [kerchunk library](https://fsspec.github.io/kerchunk/) for example builds a virtual __chunked__ dataset based on NetCDF files, and optimizes the access and analysis of large datasets.

### Data

In this workshop, we will be using the [SeasFire Data Cube](https://opensciencedata.esa.int/products/seasfire-cube/collection) published to the EarthCODE Open Science Catalog

#### Related publications
* *Alonso, Lazaro, Gans, Fabian, Karasante, Ilektra, Ahuja, Akanksha, Prapas, Ioannis, Kondylatos, Spyros, Papoutsis, Ioannis, Panagiotou, Eleannna, Michail, Dimitrios, Cremer, Felix, Weber, Ulrich, & Carvalhais, Nuno. (2022). SeasFire Cube: A Global Dataset for Seasonal Fire Modeling in the Earth System (0.4) [Data set]. Zenodo. @alonso-2024. The same dataset can also be downloaded from Zenodo: https://zenodo.org/records/13834057*


In [3]:
http_url = "https://s3.waw4-1.cloudferro.com/EarthCODE/OSCAssets/seasfire/seasfire_v0.4.zarr/"

ds = xarray.open_dataset(
	http_url,
	engine='zarr',
    chunks={},
	consolidated=True
	# storage_options = {'token': 'anon'}
)
ds

<xarray.Dataset> Size: 164GB
Dimensions:                         (latitude: 720, longitude: 1440, time: 966)
Coordinates:
  * latitude                        (latitude) float64 6kB 89.88 ... -89.88
  * longitude                       (longitude) float64 12kB -179.9 ... 179.9
  * time                            (time) datetime64[ns] 8kB 2001-01-01 ... ...
Data variables: (12/59)
    area                            (latitude, longitude) float32 4MB dask.array<chunksize=(180, 360), meta=np.ndarray>
    biomes                          (latitude, longitude) float32 4MB dask.array<chunksize=(180, 360), meta=np.ndarray>
    cams_co2fire                    (time, latitude, longitude) float32 4GB dask.array<chunksize=(966, 180, 360), meta=np.ndarray>
    cams_frpfire                    (time, latitude, longitude) float32 4GB dask.array<chunksize=(966, 180, 360), meta=np.ndarray>
    drought_code_max                (time, latitude, longitude) float32 4GB dask.array<chunksize=(966, 180, 360), meta=np.ndarray>
    drought_code_mean               (time, latitude, longitude) float32 4GB dask.array<chunksize=(966, 180, 360), meta=np.ndarray>
    ...                              ...
    t2m_max                         (time, latitude, longitude) float32 4GB dask.array<chunksize=(966, 180, 360), meta=np.ndarray>
    t2m_mean                        (time, latitude, longitude) float32 4GB dask.array<chunksize=(966, 180, 360), meta=np.ndarray>
    t2m_min                         (time, latitude, longitude) float32 4GB dask.array<chunksize=(966, 180, 360), meta=np.ndarray>
    tp                              (time, latitude, longitude) float32 4GB dask.array<chunksize=(966, 180, 360), meta=np.ndarray>
    vpd                             (time, latitude, longitude) float32 4GB dask.array<chunksize=(966, 180, 360), meta=np.ndarray>
    ws10                            (time, latitude, longitude) float32 4GB dask.array<chunksize=(966, 180, 360), meta=np.ndarray>
Attributes:
    crs:          EPSG:4326
    description:  The SeasFire Cube is a scientific datacube for seasonal fir...
    title:        SeasFire Cube: A Global Dataset for Seasonal Fire Modeling ...

In [4]:
total_size_GB = sum(var.nbytes for var in ds.data_vars.values()) / 1e9  # B to GB
print(f"Total size: {total_size_GB:.2f} GB")

Total size: 164.27 GB


## What is a __chunk__

If you look carefully at our dataset, each Data Variable is a `ndarray` (or `dask.array` if you have a dask client instantiated) with a chunk size of `(966, 180, 360)`. So basically accessing one data variable would load arrays of dimensions `(966, 180, 360)` into the computer's RAM. You can see this information and more details by clicking the icon as indicated in the image below.

![Dask.array](../static/datasize.png)

When you need to analyze large files, a computer's memory may not be sufficient anymore.

This is where understanding and using chunking correctly comes into play.

__Chunking__ is splitting a dataset into small pieces. 

Original dataset is in one piece,  
![Dask.array](../static/notchunked.png)

and we split it into several smaller pieces.  
![Dask.array](../static/chunked.png)

We split it into pieces so that we can process our data block by block or __chunk__ by __chunk__.

In our case, the data is already chunked, in a cloud-native format ready for analysis and usage - in **zarr format**


# Zarr

Zarr's main characteristics are the following:

- Every chunk of a Zarr dataset is stored as a single file
- Each Data array in a Zarr dataset has a two unique files containing metadata:
  - .zattrs for dataset or dataarray general metadatas
  - .zarray indicating how the dataarray is chunked, and where to find them on disk or other storage.
  
Zarr can be considered as an Analysis Ready, cloud optimized data (ARCO) file format!